In [0]:

%fs ls /FileStore/tables/retail-logs-data/

path,name,size
dbfs:/FileStore/tables/retail-logs-data/_spark_metadata/,_spark_metadata/,0


In [0]:
retail_logs_data = spark.read.csv("dbfs:/FileStore/tables/retail-logs-check/")

In [0]:
%sh telnet ec2-13-233-76-185.ap-south-1.compute.amazonaws.com 9092

Trying 13.233.76.185...
Connected to ec2-13-233-76-185.ap-south-1.compute.amazonaws.com.
Escape character is '^]'.

In [0]:
spark

Out[16]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:

source_df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "ec2-13-233-76-185.ap-south-1.compute.amazonaws.com:9092").option("subscribe", "retail").option("startingOffsets", "latest").option("minPartitions", "2").option("failOnDataLoss", "true").load()


In [0]:
source_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

Out[7]: DataFrame[key: string, value: string]

In [0]:

query = source_df.writeStream.trigger(processingTime='5 seconds').format("csv").outputMode("append").option("path", "/FileStore/tables/retail-logs-data").option("checkpointLocation", "/FileStore/tables/retail-logs-check").start()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-482302582970445> in <module> 
 ----> 1 query = source_df . writeStream . trigger ( processingTime = '5 seconds' ) . format ( "csv" ) . outputMode ( "append" ) . option ( "path" , "/FileStore/tables/retail-logs-data" ) . option ( "checkpointLocation" , "/FileStore/tables/retail-logs-check" ) . start ( ) 

 /databricks/spark/python/pyspark/sql/streaming.py in start (self, path, format, outputMode, partitionBy, queryName, **options) 
 1515 self . queryName ( queryName ) 
 1516 if path is None : 
 -> 1517 return self . _sq ( self . _jwrite . start ( ) ) 
 1518 else : 
 1519 return self . _sq ( self . _jwrite . start ( path ) ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o629.start.
: java.util.concurrent.TimeoutException: Stream Execution thread for stream [id = 57e32605-3bbc-4681-9d6b-e4502749b8e1, runId = 04e0d839-2f00-4416-9ee6-88fa87133bf8] failed to stop within 15000 milliseconds (specified by spark.sql.streaming.stopTimeout). See the cause on what was being executed in the streaming query thread.
	at org.apache.spark.sql.execution.streaming.StreamExecution.interruptAndAwaitExecutionThreadTermination(StreamExecution.scala:509)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution.stop(MicroBatchExecution.scala:182)
	at org.apache.spark.sql.streaming.StreamingQueryManager.$anonfun$startQuery$7(StreamingQueryManager.scala:435)
	at org.apache.spark.sql.streaming.StreamingQueryManager.$anonfun$startQuery$7$adapted(StreamingQueryManager.scala:433)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.streaming.StreamingQueryManager.startQuery(StreamingQueryManager.scala:433)
	at org.apache.spark.sql.streaming.DataStreamWriter.startQuery(DataStreamWriter.scala:498)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:488)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:306)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: The stream thread was last executing:
	at sun.nio.ch.EPollArrayWrapper.epollWait(Native Method)
	at sun.nio.ch.EPollArrayWrapper.poll(EPollArrayWrapper.java:269)
	at sun.nio.ch.EPollSelectorImpl.doSelect(EPollSelectorImpl.java:93)
	at sun.nio.ch.SelectorImpl.lockAndDoSelect(SelectorImpl.java:86)
	at s

In [0]:
query = source_df.writeStream.trigger(processingTime='5 seconds').format("console").start()

query.awaitTermination()